In [1]:
import time
start1 = time.time()
from demo2_utils import *
from crowning2_utils import *
import time
print(int(time.time()-start1),'s')

1 s


In [2]:
# get 30,31,32 tooth num cases
start = time.time()
target_path_list = []
ct = 0
for i,case in enumerate(os.listdir(udx_dirpath)):
    ct += 1
    case_path = os.path.join(udx_dirpath, case)
    my_case_details = get_toothnum(case_path)
    if not my_case_details:
        print(f'To remove: {case_path}')
        # os.remove(case_path) 
        continue
    if my_case_details[1] in [30, 31,32]:
    # if str(my_case_details[1]) == '1':
        target_path_list.append(case_path)

print(f'Total cases: {len(os.listdir(udx_dirpath))}\nFiltered cases: {len(target_path_list)}')
print(f'Runtime: {int(time.time()-start)}s')
target_idx = -1

get_toothnum Exception: File is not a zip file
To remove: /Users/nimbyx/Projects/CROWNING/cases/20230905113623089_2748284_W142B.zip
get_toothnum Exception: File is not a zip file
To remove: /Users/nimbyx/Projects/CROWNING/cases/.DS_Store
Total cases: 267
Filtered cases: 55
Runtime: 97s


In [ ]:
#PROCESS
# target_idx += 1

start1 = time.time()
file_path = target_path_list[target_idx]
mesh_prep_fname, mesh_prep_3m = load_prep_from_zip(file_path)
mesh_anta_fname, mesh_anta_3m = load_anta_from_zip(file_path)
mesh_big, mesh_small = get_two_largest_meshes_3m(mesh_prep_3m)
my_case_details = get_toothnum(file_path)
tooth_filepath = os.path.join(tooth_dirpath_ottawa, f"{my_case_details[1]}.stl")
tooth_mesh = pv.read(tooth_filepath)
tooth_desi = get_designer_tooth(file_path)

# case when z plane is larger than x plane
largest_plane = get_largest_axis_idx(mesh_prep_3m.bounds)
print(f'largest plane: {largest_plane}')
if largest_plane in [0,2]:
    if largest_plane == 2:
        tooth_mesh1 = get_horizontal_position(mesh_prep_3m.bounds, mesh_small.centroid, tooth_mesh)
    elif largest_plane == 0:
        tooth_mesh1 = get_vertical_position(mesh_prep_3m.bounds, mesh_small.centroid, tooth_mesh)
    tooth_mesh2 = tooth_mesh1.scale(np.full(3, 1.20))
    tooth_mesh3 = tooth_mesh2.translate(gen_move_coords(mesh_small.centroid, tooth_mesh2.center))
    my_crown = tooth_mesh3.boolean_difference(pv.wrap(mesh_small))
   
    cam_pos = 'xy'
    my_shape = (1,4)
    p1_ = pv.Plotter(shape=my_shape)
        
    p1_.subplot(0,0) #1st plot
    p1_.add_mesh(mesh_small)
    p1_.add_mesh(mesh_big)
    p1_.add_mesh(tooth_mesh3, 
                 color='gold')
    p1_.add_text(f'{my_case_details[0]}', position='lower_right', font_size=10)
    p1_.camera_position=cam_pos
    p1_.show_grid()

    p1_.subplot(0,1) #2nd plot
    p1_.add_mesh(pv.wrap(mesh_anta_3m), opacity=.8)
    # p1_.add_mesh(tooth_mesh3.translate(gen_move_coords(tooth_mesh3.center-np.array([0,0,1]), tooth_mesh3.center)), color='w')
    p1_.add_mesh(tooth_mesh3, color='gold')
    p1_.add_text(f'{my_case_details[1]}', position='upper_right')
    p1_.add_text('antagonist', position='lower_right', font_size=10)
    p1_.camera_position=cam_pos
    
    p1_.subplot(0,2) #3rd plot
    # p1_.add_mesh(my_crown, color='gold', opacity=.9)
    # p1_.add_points(tooth_edges, color='red', line_width=10)
    p1_.add_mesh(pv.wrap(mesh_small))
    p1_.add_text('auto', position='lower_right', font_size=10)
    # p1_.enable_surface_point_picking(callback=callback2, show_point=True)
    p1_.show_grid()
    p1_.camera_position=cam_pos

    p1_.subplot(0,3) #4th plot
    p1_.add_mesh(pv.wrap(mesh_small))
    p1_.add_mesh(tooth_desi, color='w', opacity=.9)
    p1_.add_text('designer', position='lower_right', font_size=10)
    p1_.show_grid()
    p1_.camera_position=cam_pos

    p1_.show()
else:
    print('y-plane largest')
print(int(time.time()-start1), 's')

In [54]:
#PROCESS v2
# target_idx += 1

start1 = time.time()
file_path = target_path_list[target_idx]
mesh_prep_fname, mesh_prep_3m = load_prep_from_zip(file_path)
mesh_anta_fname, mesh_anta_3m = load_anta_from_zip(file_path)
mesh_big, mesh_small = get_two_largest_meshes_3m(mesh_prep_3m)
my_case_details = get_toothnum(file_path)
tooth_filepath = os.path.join(tooth_dirpath_ottawa, f"{my_case_details[1]}.stl")
tooth_mesh = pv.read(tooth_filepath)
tooth_desi = get_designer_tooth(file_path)

# get margin
margin = pv.wrap(mesh_small).extract_feature_edges(5).connectivity().extract_largest()
base_bot = pv.wrap(mesh_small).extract_feature_edges(90)
if not margin:
    print('Margin error')
    
elif base_bot.center == margin.center:
    print('Unable to detect margin')
    
else:
    # case when z plane is larger than x plane
    largest_plane = get_largest_axis_idx(mesh_prep_3m.bounds)
    print(f'largest plane: {largest_plane}')
    if largest_plane in [0,2]:
        if largest_plane == 2:
            tooth_mesh1 = get_horizontal_position(mesh_prep_3m.bounds, mesh_small.centroid, tooth_mesh)
        elif largest_plane == 0:
            tooth_mesh1 = get_vertical_position(mesh_prep_3m.bounds, mesh_small.centroid, tooth_mesh)
        # tooth_mesh1 = tooth_mesh1.rotate_z(-compute_angle2(mesh_small))
        
        tooth_mesh2 = tooth_mesh1.scale(np.full(3, 1.00))
        # tooth_mesh3 = tooth_mesh2.translate(gen_move_coords(mesh_small.centroid, tooth_mesh2.center))
        tooth_mesh3 = tooth_mesh2.translate(gen_move_coords(margin_center, tooth_mesh2.center))
        my_crown = tooth_mesh3.boolean_difference(pv.wrap(mesh_small)).fill_holes(5)
       
        cam_pos = 'xy'
        my_shape = (1,3)
        p1_ = pv.Plotter(shape=my_shape)
            
        p1_.subplot(0,0) #1st plot
        p1_.add_mesh(mesh_small)
        p1_.add_mesh(mesh_big)
        p1_.add_mesh(tooth_mesh3, 
                     color='gold')
        p1_.add_text(f'{my_case_details[0]}', position='lower_right', font_size=10)
        p1_.camera_position=cam_pos
        p1_.show_grid()
    
        p1_.subplot(0,1) #2nd plot
        p1_.add_mesh(pv.wrap(mesh_anta_3m), opacity=.8)
        # p1_.add_mesh(tooth_mesh3.translate(gen_move_coords(tooth_mesh3.center-np.array([0,0,1]), tooth_mesh3.center)), color='w')
        p1_.add_mesh(tooth_mesh3, color='gold')
        p1_.add_text(f'{my_case_details[1]}', position='upper_right')
        p1_.add_text('antagonist', position='lower_right', font_size=10)
        p1_.camera_position=cam_pos
        
        p1_.subplot(0,2) #3rd plot
        p1_.add_mesh(my_crown, color='gold')
        # p1_.add_points(tooth_edges, color='red', line_width=10)
        # p1_.add_mesh(pv.wrap(mesh_small))
        p1_.add_text('auto', position='lower_right', font_size=10)
        # p1_.enable_surface_point_picking(callback=callback2, show_point=True)
        p1_.show_grid()
        p1_.camera_position=cam_pos
    
        # p1_.subplot(0,3) #4th plot
        # p1_.add_mesh(pv.wrap(mesh_small))
        # p1_.add_mesh(tooth_desi, color='w', opacity=.9)
        # p1_.add_text('designer', position='lower_right', font_size=10)
        # p1_.show_grid()
        # p1_.camera_position=cam_pos
    
        p1_.show()
    else:
        print('y-plane largest')
    print(int(time.time()-start1), 's')

largest plane: 0
LOWER


Widget(value="<iframe src='http://localhost:50218/index.html?ui=P_0x326adb2b0_42&reconnect=auto' style='width:…

1 s


In [12]:
base_edges = pv.wrap(mesh_small).extract_feature_edges(90)

In [55]:
p2 = pv.Plotter(shape=(1,2))
p2.subplot(0,0)
p2.add_mesh(my_crown.smooth(), color='gold')
p2.show_grid()
p2.camera_position=cam_pos

p2.subplot(0,1)
p2.add_mesh(pv.wrap(mesh_small).smooth(), color='pink')#, opacity=0.8)
p2.add_points(np.array(margin_center), color='r', point_size=15, render_points_as_spheres=True)
p2.show_grid()
p2.camera_position=cam_pos

p2.show()

Widget(value="<iframe src='http://localhost:50218/index.html?ui=P_0x3917340d0_43&reconnect=auto' style='width:…

In [38]:
margin_center = np.array(pv.wrap(mesh_small).extract_feature_edges(12).connectivity().extract_largest().center)
margin_center

array([-2.3597244 , 10.68001509, -1.90782607])

In [ ]:
pv.wrap(mesh_small).bounds

In [ ]:
my_opp = abs(base_edges.center[1] - pv.wrap(mesh_small).bounds[2]) 
my_adj = abs(base_edges.center[0] - pv.wrap(mesh_small).bounds[1])

In [ ]:
import math

# Length of the adjacent side (e.g., 3.0 units)
adjacent_length = my_adj

# Length of the opposite side (e.g., 4.0 units)
opposite_length = my_opp

# Calculate the angle in radians
angle_radians = math.atan(opposite_length / adjacent_length)

# Convert the angle from radians to degrees
angle_degrees = math.degrees(angle_radians)
angle_degrees

In [ ]:
compute_angle(lowest_point, base_edges.center)

# WORKSPACE

In [ ]:
edges = pv.wrap(mesh_small).extract_feature_edges(15).connectivity().extract_largest()
edges2 = pv.wrap(mesh_small).extract_feature_edges(15)
p4 = pv.Plotter()
p4.add_points(np.array(edges.center), point_size=15,  color='green', render_points_as_spheres=True)
p4.add_mesh(edges, color='gold')
p4.add_mesh(edges2, color='r')


p4.show()


# CASE VIEWER PTS

In [ ]:
import json

tooth_num = 4
txt_path = '/Users/nimbyx/Downloads/2023-09-07T02_21_39.005Z.txt'
with open(txt_path, 'r') as f:
    my_pts = json.load(f)

pt1 = string_to_numpy_array(my_pts['points'][0]['firstPoint'])
pt2 = string_to_numpy_array(my_pts['points'][0]['secondPoint'])

tooth_filepath = os.path.join(tooth_dirpath_ottawa, f"{tooth_num}.stl")
tooth4 = pv.read(tooth_filepath)

p3 = pv.Plotter()
p3.add_points(np.array(pt1), color='r', point_size=15, render_points_as_spheres=True)
p3.add_points(np.array(pt2), color='blue', point_size=15, render_points_as_spheres=True)
p3.add_mesh(tooth4)

p3.show_grid()
p3.show()